In [1]:
!pip install spacy --user
!python3 -m spacy download it_core_news_sm
!pip3 install -U spacy
!pip3 install spacy
!python -m spacy download it_core_news_sm

     |████████████████████████████████| 21.1 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')
  Using cached https://github.com/explosion/spacy-models/releases/download/it_core_news_sm-3.1.0/it_core_news_sm-3.1.0-py3-none-any.whl (21.1 MB)
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [2]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import gensim
from gensim.models import Word2Vec
import pandas as pd

In [3]:
df= pd.read_csv('Books.csv')
df

,Unnamed: 0,_id,author,contents,publishing house,year
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
df['Frasi']= df.apply(lambda row: nltk.sent_tokenize(row['contents']), axis=1)
df

,Unnamed: 0,_id,author,contents,publishing house,year,Frasi
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ..."


In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import spacy
from spacy.lang.it.examples import sentences 
from tqdm.notebook import tqdm
tqdm.pandas()
nlp = spacy.load('it_core_news_sm')
nlp.max_length = 1064150 
pipe = nlp.pipe(df['contents'], n_process=2, batch_size=2000)

In [8]:
docs = [x for x in tqdm(pipe)]

0it [00:00, ?it/s]

In [9]:
df['spacy_doc'] = docs

In [10]:
df['POS_spacy'] = df['spacy_doc'].progress_apply(lambda x: [(y.text, y.pos_) for y in x])

  0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
df['lemmatized'] = df['spacy_doc'].progress_apply(lambda x: " ".join([y.lemma_ for y in x if len(x)>1]))

  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
def filter_text_pos(x):
    final_pos_text = []
    for elem in x:
        for pos in pos_list:
            if elem[1] == pos:
                final_pos_text.append(elem[0])
    
    return " ".join(final_pos_text)

In [13]:
import nltk
from nltk.corpus import stopwords
import string
nltk.download('stopwords')
def clean(x):
  stop = stopwords.words("italian")
  stop = stop + list(string.punctuation)
  words  = [w for w in word_tokenize(x.lower()) if w.isalpha() and w not in stop]
  return " ".join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
def clean(x):
  stop = stopwords.words("italian")
  stop = stop + list(string.punctuation)
  words  = [w for w in word_tokenize(x.lower()) if w.isalpha() and w not in stop]
  return " ".join(words)

In [15]:
df

,Unnamed: 0,_id,author,contents,publishing house,year,Frasi,spacy_doc,POS_spacy,lemmatized
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...,"(Quel, giorno, era, impossibile, uscire, a, pa...","[(Quel, DET), (giorno, NOUN), (era, AUX), (imp...",quello giorno essere impossibile uscire a pass...
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ...","(In, quel, giorno, era, impossibile, passeggia...","[(In, ADP), (quel, DET), (giorno, NOUN), (era,...",in quello giorno essere impossibile passeggiar...


In [16]:
df['Frasi2']= df.apply(lambda row: nltk.sent_tokenize(row['lemmatized']), axis=1)
df

,Unnamed: 0,_id,author,contents,publishing house,year,Frasi,spacy_doc,POS_spacy,lemmatized,Frasi2
0,0,Jane Eyre_Feltrinelli_pulito.txt,Charlotte Brontë,Quel giorno era impossibile uscire a passeggio...,Feltrinelli Editore,2014,[Quel giorno era impossibile uscire a passeggi...,"(Quel, giorno, era, impossibile, uscire, a, pa...","[(Quel, DET), (giorno, NOUN), (era, AUX), (imp...",quello giorno essere impossibile uscire a pass...,[quello giorno essere impossibile uscire a pas...
1,1,Jane Eyre_Treves_pulito.txt,Charlotte Brontë,In quel giorno era impossibile passeggiare. La...,Fratelli Treves Editori,1904,"[In quel giorno era impossibile passeggiare., ...","(In, quel, giorno, era, impossibile, passeggia...","[(In, ADP), (quel, DET), (giorno, NOUN), (era,...",in quello giorno essere impossibile passeggiar...,[in quello giorno essere impossibile passeggia...


In [17]:
import numpy as np
documents = list(np.array(df['Frasi2'][0]))
documents2 = list(np.array(df['Frasi2'][1]))

df = pd.DataFrame(documents,columns={'text'})
df2 = pd.DataFrame(documents2, columns={'text'})
df3 = df.append(df2)

In [19]:
df["Lemmi"] = df["text"].apply(clean)
df2["Lemmi"] = df2["text"].apply(clean)
df3["Lemmi"] = df3["text"].apply(clean)

In [20]:
df3["Lemmi"]

0              giorno essere impossibile uscire passeggio
1       mattino realtà avere gironzolare ora arbusto s...
2                                           essere felice
3       lunghe passeggiate essere mai piacere specie p...
4       essere terribile tornare casa pungente freddo ...
                              ...                        
8224    sapere prossimo volta scrivere estraneo annunz...
8225    piangere ultimo ora essere turbare timore mort...
8226    profezia attestare egli scrivere maestro avere...
8227    avanzo rapidamente ogni ora scorrere risponder...
8228                                                 fine
Name: Lemmi, Length: 19802, dtype: object

In [22]:
df3['documents_cleaned']= df3.apply(lambda row: nltk.word_tokenize(row['Lemmi']), axis=1)
data = list(df3['documents_cleaned'])

In [24]:
# Create Skip Gram model
model2 = gensim.models.Word2Vec(data, min_count = 1, size = 50, window = 5, sg = 1)

In [25]:
model2.init_sims(replace=True)

In [26]:
model2.wv.most_similar(positive=["giorno"])

[('settimana', 0.949376106262207),
 ('mese', 0.9392577409744263),
 ('fa', 0.9344190955162048),
 ('minuto', 0.9223361015319824),
 ('cinque', 0.9204700589179993),
 ('quindici', 0.9187092781066895),
 ('dieci', 0.9182727336883545),
 ('trascorrere', 0.9182222485542297),
 ('sera', 0.9137798547744751),
 ('anno', 0.9117920994758606)]

In [27]:
model2.wv.similarity('giorno','mattino')

0.8992375

In [28]:
similar = model2.wv.most_similar('mattino')
similar

[('mezz', 0.9910297393798828),
 ('pomeriggio', 0.9894804358482361),
 ('paio', 0.9894293546676636),
 ('mattina', 0.9883600473403931),
 ('trascorrere', 0.9880746603012085),
 ('durante', 0.9853440523147583),
 ('riflettere', 0.9847015738487244),
 ('silenzio', 0.9834097027778625),
 ('notte', 0.9826017022132874),
 ('quarto', 0.9821746349334717)]

In [29]:
model2.init_sims(replace = True)
model2.save('word2vec_model_lemmi')
model = Word2Vec.load('word2vec_model_lemmi')
model.wv.most_similar('pomeriggio')

[('studio', 0.9919933676719666),
 ('mattino', 0.9894805550575256),
 ('passeggiare', 0.9884764552116394),
 ('baule', 0.9880860447883606),
 ('quarto', 0.9879050850868225),
 ('paio', 0.9877115488052368),
 ('quattro', 0.9849091172218323),
 ('rimare', 0.984549880027771),
 ('scorso', 0.9843419790267944),
 ('mezzanotte', 0.9840087890625)]

In [30]:
model.wv.most_similar('mattino')

[('mezz', 0.9910297393798828),
 ('pomeriggio', 0.9894805550575256),
 ('paio', 0.9894293546676636),
 ('mattina', 0.9883600473403931),
 ('trascorrere', 0.988074779510498),
 ('durante', 0.9853440523147583),
 ('riflettere', 0.9847017526626587),
 ('silenzio', 0.9834098815917969),
 ('notte', 0.9826017022132874),
 ('quarto', 0.9821746349334717)]

In [32]:
data[0]

['giorno', 'essere', 'impossibile', 'uscire', 'passeggio']

In [31]:
distance = model.wv.wmdistance(data[0],data[12])
distance

0.5852359330931124

In [36]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.021861297333683125

In [37]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'
distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

inf

In [38]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.013078101703717213

In [39]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model.wv.wmdistance(sentence_a,sentence_b)
distance

0.04566559224448453

#DOC2VEC

In [40]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
tagged_data = [TaggedDocument(words=word_tokenize(doc), tags=[i]) for i, doc in enumerate(df3["Lemmi"])]
model_d2v = Doc2Vec(vector_size=5, dbow_words=1, dm=0, epochs=30,  window=2, seed=1337, min_count=1, alpha=0.001, min_alpha=0.025)
model_d2v.build_vocab(tagged_data)
model_d2v.save('doc2vec_model')

In [41]:
model_d2v = Doc2Vec(vector_size=5, window=2, min_count=1, workers=4)
model_d2v.build_vocab(tagged_data)

In [42]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(15271, 0.9918297529220581),
 (7718, 0.9912025332450867),
 (7452, 0.9890585541725159),
 (11498, 0.9883577227592468),
 (11644, 0.9876433610916138),
 (2337, 0.9876022338867188),
 (12680, 0.9868730306625366),
 (5164, 0.986814558506012),
 (6148, 0.9866076707839966),
 (19639, 0.984277606010437)]

In [47]:
tagged_data[15271]

TaggedDocument(words=['sì', 'lambiccare', 'cervello', 'conto', 'dimenticare'], tags=[15271])

In [43]:
tokens = 'Quel giorno era impossibile uscire a passeggio.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(2027, 0.9984607696533203),
 (6115, 0.9938331842422485),
 (6562, 0.9920634031295776),
 (10793, 0.9917377233505249),
 (4461, 0.9889070391654968),
 (13473, 0.983848512172699),
 (6547, 0.9821035861968994),
 (12440, 0.9792963266372681),
 (14099, 0.9792187213897705),
 (10594, 0.9791697263717651)]

In [48]:
tagged_data[2027]

TaggedDocument(words=['meglio', 'così'], tags=[2027])

In [49]:
tokens = 'In quel giorno era impossibile passeggiare.'
new_vector = model_d2v.infer_vector(tokens)
sims = model_d2v.docvecs.most_similar([new_vector]) 
sims

[(15271, 0.9918297529220581),
 (7718, 0.9912025332450867),
 (7452, 0.9890585541725159),
 (11498, 0.9883577227592468),
 (11644, 0.9876433610916138),
 (2337, 0.9876022338867188),
 (12680, 0.9868730306625366),
 (5164, 0.986814558506012),
 (6148, 0.9866076707839966),
 (19639, 0.984277606010437)]

In [50]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.01599083720937653

In [51]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

inf

In [52]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.009807271120712405

In [53]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

distance = model_d2v.wv.wmdistance(sentence_a,sentence_b)
distance

0.014466220047621853

In [54]:
sentence_a = 'Quel giorno era impossibile uscire a passeggio.'
sentence_b = 'In quel giorno era impossibile passeggiare.'
new_vector_a = model_d2v.infer_vector(sentence_a)
new_vector_b = model_d2v.infer_vector(sentence_b)

cosine_similarity = np.dot(new_vector_a, new_vector_b)/(np.linalg.norm(new_vector_a)* np.linalg.norm(new_vector_b))
cosine_similarity

-0.04780282

SENT2VEC

In [55]:
!pip install bert
!pip install pytorch_pretrained_bert
!pip install sent2vec
!pip install sentence_transformers

In [56]:
import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import random
import numpy as np
import matplotlib.pyplot as plt
import math

In [57]:
import torch#pytorch
from transformers import AutoTokenizer, AutoModel#for embeddings
from sklearn.metrics.pairwise import cosine_similarity#for similarity

#download pretrained model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-italian-cased",output_hidden_states=True)

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [58]:
#create embeddings
def get_embeddings(text,token_length):
  tokens=tokenizer(text,max_length=token_length,padding='max_length',truncation=True)
  output=model(torch.tensor(tokens.input_ids).unsqueeze(0),
               attention_mask=torch.tensor(tokens.attention_mask).unsqueeze(0)).hidden_states[-1]
  return torch.mean(output,axis=1).detach().numpy()

In [59]:
#calculate similarity
def calculate_similarity(text1,text2,token_length=20):
    out1=get_embeddings(text1,token_length=token_length)#create embeddings of text
    out2=get_embeddings(text2,token_length=token_length)#create embeddings of text
    sim= cosine_similarity(out1,out2)[0][0]
    print(sim)

In [60]:
calculate_similarity(df['Lemmi'][0],df2['Lemmi'][0])

0.9500569


In [61]:
sentence_a = 'Ne ero contenta.'
sentence_b = 'Ne fui felice.'

calculate_similarity(sentence_a,sentence_b)

0.9024403


In [62]:
sentence_a = 'Quanto a me, mi aveva dispensata dall onere di unirmi al gruppo, dicendo che le rincresceva di dovermi tenere a distanza, ma finchÈ non avesse saputo da Bessie e visto con i suoi occhi che facevo seri sforzi per assumere un atteggiamento più socievole e adatto a una bambina scarlatta e a sinistra c erano le vetrate trasparenti che mi proteggevano, senza separarmene, da quella triste giornata di novembre.'
sentence_b ='Ella mi aveva proibito di unirmi al loro gruppo, dicendo che deplorava la necessità in cui trovavasi di tenermi così lontana, ma che fino al momento in cui Bessie non guarentirebbe che mi studiavo di acquistare un carattere pi˘ socievole e pi˘ infantile, maniere pi˘ cortesi e qualcosa di pi˘ radioso, di più aperto, di più sincero, non poteva concedermi gli stessi privilegi che ai bambini allegri e soddisfatti.'

calculate_similarity(sentence_a,sentence_b)

0.94164723


In [63]:
sentence_a = 'Non mi sono mai piaciute le lunghe passeggiate, sopra tutto col freddo, ed era cosa penosa per me di tornar di notte con le mani e i piedi gelati, col cuore amareggiato dalle sgridate di Bessie, la bambinaia, e con lo spirito abbattuto dalla coscienza della mia inferiorità fisica di fronte a Eliza, a John e a Georgiana Reed.'

sentence_b = 'Era terribile tornare a casa nel pungente freddo del crepuscolo, con mani e piedi gelati e il cuore intristito dai rimproveri di Bessie, la balia, e avvilita dalla consapevolezza della mia inferiorità fisica rispetto a Eliza, John e Georgiana Reed. '

calculate_similarity(sentence_a,sentence_b)

0.80446357
